In [5]:
import numpy as np
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [6]:
url = 'https://www.realestate.com.au/international/us/rent'

browser = webdriver.Chrome()
browser.get(url)
browser.implicitly_wait(20)

In [7]:
houses = pd.DataFrame(columns=[
    'Price', 
    'Rooms',
    'Type',
    'Status',
    'Building Size',
    'Architecture Style', 
    'County',
    'Provider',
])

In [ ]:
while (len(houses) < 1000):
    links = browser.find_elements(By.XPATH, '//div[@class="sc-1dun5hk-0 cOiOrj"]/a')

    urls = []
    for link in links:
        urls.append(link.get_attribute('href'))

    for sub_url in urls:
        browser.get(sub_url)
        price = browser.find_element(By.XPATH, '//div[@class="sc-10v3xoh-1 cqrlhJ"]').text
        info_list = browser.find_elements(By.XPATH, '//div[contains(@class, "sc-12iqlu8-0")]')
        
        basic_info = {}
        
        for info in info_list:
            key = info.find_element(By.XPATH, './/div[@class="sc-12iqlu8-1 Mguuo basicInfoKey"]').text
            value = info.find_element(By.XPATH, './/div[@class="sc-12iqlu8-2 dPeBQf basicInfoValue"]').text
            
            basic_info[key] = value
        
        browser.back()    

        house = pd.Series([
            price,
            basic_info.get('Rooms', None),
            basic_info.get('Property Type', None),
            basic_info.get('Listing Status', None),
            basic_info.get('Building Size', None),
            basic_info.get('Architecture Style', None),
            basic_info.get('County', None),
            basic_info.get('Provider', None)
        ], index=['Price', 'Rooms', 'Type', 'Status', 'Building Size', 'Architecture Style', 'County', 'Provider'])
        
        houses.loc[len(houses)] = house
        
    
    next_page = browser.find_element(By.XPATH, '//li[@title="Next Page"]/a')
    next_url = next_page.get_attribute('href')
    
    browser.get(next_url)
    
houses